<h1> Magic Formula - Numerofobia</h1>
---

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import yfinance as yf

In [4]:
# função para converter valores percentuais para valores decimais
def convert_perc(value):
    return pd.to_numeric(value.replace('%','').replace('.', '').replace(',', '.'))

# mapeando colunas que possuem valores percentuais
convs = {5: convert_perc, 12: convert_perc, 13: convert_perc, 15: convert_perc, 16: convert_perc, 20: convert_perc}

url = 'http://fundamentus.com.br/resultado.php'
agent = {"User-Agent":"Mozzila/5.0"}
resposta = requests.get(url, headers = agent)
soup = BeautifulSoup(resposta.text, 'lxml')
tabela = soup.find_all('table')[0]
df = pd.read_html(str(tabela), decimal = ',', thousands='.', converters=convs)[0]
#df = pd.read_html(str(tabela), decimal = ',', thousands='.',index_col='Papel', converters=convs)[0]
"""df.columns.name = df.index.name
df.index.name = 'Papel'"""
fundamentus = df
fundamentus

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,MNSA4,0.47,0.00,0.00,0.000,0.0,0.000,0.0,0.00,0.00,...,0.00,-208.15,-362.66,3.63,-13.50,145.70,0.0,-9.105000e+06,-6.52,-41.11
1,CSTB4,147.69,0.00,0.00,0.000,0.0,0.000,0.0,0.00,0.00,...,0.00,40.85,28.98,2.60,22.40,20.11,0.0,8.420670e+09,0.14,31.91
2,PORP4,2.40,0.00,0.00,0.000,0.0,0.000,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,-2.08,0.0,2.239900e+07,0.00,13.66
3,CFLU4,1000.00,0.00,0.00,0.000,0.0,0.000,0.0,0.00,0.00,...,0.00,8.88,10.72,1.10,17.68,32.15,0.0,6.035100e+07,0.06,8.14
4,CLAN3,0.00,0.00,0.00,0.000,0.0,0.000,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,-1.05,0.0,1.012240e+09,0.00,-63.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,UBBR4,7.49,610.27,1.99,0.000,0.0,0.000,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.33,0.0,1.031720e+10,0.00,10.58
982,UBBR11,14.75,1201.81,3.91,0.000,0.0,0.000,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.33,0.0,1.031720e+10,0.00,10.58
983,UBBR3,18.00,1466.61,4.77,0.000,0.0,0.000,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.33,0.0,1.031720e+10,0.00,10.58
984,BBTG12,3.15,29554.00,24180.60,0.000,0.0,0.000,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,81.82,0.0,1.100000e+04,0.00,0.00


In [5]:
# empresas que possuem alguma liquidez
fundamentus = fundamentus[fundamentus['Liq.2meses'] > 1000000]

# com retornos positivos
fundamentus = fundamentus[fundamentus['ROIC'] > 0]
fundamentus = fundamentus[fundamentus['EV/EBIT'] > 0]

#Maiorliquidez
fundamentus = fundamentus[fundamentus['Patrim. Líq'] > 0]

fundamentus.describe()

,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,2.050000e+02,2.050000e+02,205.000000,205.000000
mean,17.737268,-1.376293,2.001366,1.955234,7.561805,0.764820,-2.705268,13.329902,-7.025805,16.614683,8.543024,22.627756,17.798293,2.141268,15.808146,20.810098,9.725363e+07,1.359472e+10,1.117073,20.496537
std,15.743422,148.223148,1.658792,2.222258,39.488445,0.601261,122.628974,26.370406,111.689612,31.896702,6.928204,17.429380,26.998832,1.151165,19.406010,22.440217,2.608557e+08,4.568600e+10,2.062063,29.709466
min,1.440000,-1441.350000,0.270000,0.113000,0.000000,0.085000,-1697.940000,0.500000,-1561.410000,0.210000,0.210000,0.330000,-23.930000,0.260000,0.320000,-10.610000,1.040570e+06,1.881180e+08,0.000000,-23.950000
25%,7.740000,4.910000,0.960000,0.647000,0.620000,0.365000,1.900000,4.080000,-3.010000,5.520000,4.240000,10.200000,5.580000,1.410000,5.910000,8.120000,7.560880e+06,1.186460e+09,0.300000,5.840000
50%,13.580000,8.260000,1.610000,1.114000,2.820000,0.619000,3.490000,7.300000,-1.250000,8.770000,6.610000,17.910000,13.770000,1.840000,12.030000,16.400000,2.533770e+07,3.263940e+09,0.640000,14.390000
75%,22.300000,16.100000,2.560000,2.334000,6.010000,0.950000,7.090000,14.050000,2.210000,16.970000,11.200000,31.990000,21.470000,2.530000,18.460000,28.370000,1.022500e+08,1.017040e+10,1.340000,28.020000
max,103.200000,339.140000,15.650000,15.423000,560.940000,4.514000,209.850000,217.540000,276.760000,309.530000,44.330000,81.120000,256.490000,7.520000,232.010000,132.560000,2.491200e+09,4.341940e+11,24.270000,310.030000


In [6]:
# Criando os Rankings
"""
fundamentus['roic_rank'] = fundamentus['ROIC'].rank(method='dense', ascending=True)
fundamentus['evebit_rank'] = fundamentus['EV/EBIT'].rank(method='dense', ascending=False)
fundamentus['rank'] = fundamentus['roic_rank'] + fundamentus['evebit_rank']

# Magic Wallet
pd.DataFrame(fundamentus['rank'].nlargest(20).sort_values())
"""

"\nfundamentus['roic_rank'] = fundamentus['ROIC'].rank(method='dense', ascending=True)\nfundamentus['evebit_rank'] = fundamentus['EV/EBIT'].rank(method='dense', ascending=False)\nfundamentus['rank'] = fundamentus['roic_rank'] + fundamentus['evebit_rank']\n\n# Magic Wallet\npd.DataFrame(fundamentus['rank'].nlargest(20).sort_values())\n"

In [7]:
fundamentusNovo = fundamentus[['Papel', 'ROIC','EV/EBIT']]
fundamentusNovo

,Papel,ROIC,EV/EBIT
11,LWSA3,0.71,122.49
12,ELMD3,4.90,47.46
16,BRPR3,2.02,30.30
18,RAIL3,4.47,23.82
34,AALR3,3.28,39.54
...,...,...,...
966,MGLU3,0.95,122.27
971,EMBR3,4.72,10.39
972,SEER3,6.83,6.66
973,HAPV3,0.32,247.14


In [8]:
EvOrdenado = fundamentusNovo.sort_values(by = 'EV/EBIT')
EvOrdenado = EvOrdenado.reset_index(drop = True)
EvOrdenado['EvOrdenado'] = EvOrdenado.index
EvOrdenado

,Papel,ROIC,EV/EBIT,EvOrdenado
0,PSSA3,54.29,0.21,0
1,GOAU4,35.87,0.85,1
2,SYNE3,37.58,1.05,2
3,USIM3,31.46,1.30,3
4,USIM5,31.46,1.37,4
...,...,...,...,...
200,MGLU3,0.95,122.27,200
201,LWSA3,0.71,122.49,201
202,TPIS3,0.43,154.51,202
203,HAPV3,0.32,247.14,203


In [9]:
roicOrdenado = EvOrdenado.sort_values(by = 'ROIC', ascending = False)
roicOrdenado = roicOrdenado.reset_index(drop = True)
roicOrdenado['ROICOrdenado'] = roicOrdenado.index
dados = roicOrdenado
dados

,Papel,ROIC,EV/EBIT,EvOrdenado,ROICOrdenado
0,BRBI11,232.01,6.74,71,0
1,UNIP6,57.45,4.18,29,1
2,UNIP3,57.45,3.88,25,2
3,TASA3,57.32,2.83,18,3
4,TASA4,57.32,2.82,17,4
...,...,...,...,...,...
200,MGLU3,0.95,122.27,200,200
201,LWSA3,0.71,122.49,201,201
202,TPIS3,0.43,154.51,202,202
203,SEQL3,0.34,309.53,204,203


In [10]:
dados['score'] = dados['EvOrdenado'] + dados['ROICOrdenado']
dados.sort_values(by = 'score')

,Papel,ROIC,EV/EBIT,EvOrdenado,ROICOrdenado,score
5,PSSA3,54.29,0.21,0,5,5
14,SYNE3,37.58,1.05,2,14,16
11,BRKM5,39.71,2.14,7,11,18
9,MRFG3,41.81,2.29,10,9,19
18,GOAU4,35.87,0.85,1,18,19
...,...,...,...,...,...,...
200,MGLU3,0.95,122.27,200,200,400
201,LWSA3,0.71,122.49,201,201,402
202,TPIS3,0.43,154.51,202,202,404
203,SEQL3,0.34,309.53,204,203,407


In [11]:
# função para retira as empresas de finanças
def GetSector(nomeDoAtivo):
    setor = yf.Ticker(nomeDoAtivo)
    return setor.info['sector']

In [12]:
symbols = dados['Papel'] + '.SA'

sectorDict = {}

for tick in symbols:
    try:
        print("Coletando o setor do atvio: ", tick)
        sectorDict[tick] = GetSector(tick)
    except:
        print("Não deu certo para o ativo: ", tick)
        sectorDict[tick] = 'NA'

stockSectorDF = pd.DataFrame.from_dict(sectorDict, orient = 'index').reset_index()
stockSectorDF.columns = ['Papel', 'sector']

Coletando o setor do atvio:  BRBI11.SA
Coletando o setor do atvio:  UNIP6.SA
Coletando o setor do atvio:  UNIP3.SA
Coletando o setor do atvio:  TASA3.SA
Coletando o setor do atvio:  TASA4.SA
Coletando o setor do atvio:  PSSA3.SA
Coletando o setor do atvio:  CMIN3.SA
Coletando o setor do atvio:  AURA33.SA
Coletando o setor do atvio:  VALE3.SA
Coletando o setor do atvio:  MRFG3.SA
Coletando o setor do atvio:  KEPL3.SA
Coletando o setor do atvio:  BRKM5.SA
Coletando o setor do atvio:  PRIO3.SA
Coletando o setor do atvio:  ODPV3.SA
Coletando o setor do atvio:  SYNE3.SA
Coletando o setor do atvio:  FESA4.SA
Coletando o setor do atvio:  GGBR3.SA
Coletando o setor do atvio:  GGBR4.SA
Coletando o setor do atvio:  GOAU4.SA
Coletando o setor do atvio:  ETER3.SA
Coletando o setor do atvio:  CSNA3.SA
Coletando o setor do atvio:  FHER3.SA
Coletando o setor do atvio:  BLAU3.SA
Coletando o setor do atvio:  USIM5.SA
Coletando o setor do atvio:  USIM3.SA
Coletando o setor do atvio:  TFCO4.SA
Coletando 

In [13]:
dados['Papel'] = dados['Papel'] +'.SA'

In [14]:
dados

,Papel,ROIC,EV/EBIT,EvOrdenado,ROICOrdenado,score
0,BRBI11.SA,232.01,6.74,71,0,71
1,UNIP6.SA,57.45,4.18,29,1,30
2,UNIP3.SA,57.45,3.88,25,2,27
3,TASA3.SA,57.32,2.83,18,3,21
4,TASA4.SA,57.32,2.82,17,4,21
...,...,...,...,...,...,...
200,MGLU3.SA,0.95,122.27,200,200,400
201,LWSA3.SA,0.71,122.49,201,201,402
202,TPIS3.SA,0.43,154.51,202,202,404
203,SEQL3.SA,0.34,309.53,204,203,407


In [15]:
stockSectorDF

,Papel,sector
0,BRBI11.SA,Financial Services
1,UNIP6.SA,Basic Materials
2,UNIP3.SA,Basic Materials
3,TASA3.SA,Industrials
4,TASA4.SA,Industrials
...,...,...
200,MGLU3.SA,Consumer Cyclical
201,LWSA3.SA,Technology
202,TPIS3.SA,Industrials
203,SEQL3.SA,Industrials


In [20]:
dadosNovos = pd.merge(dados, stockSectorDF, on ='Papel', how='inner')
dadosNovos

,Papel,ROIC,EV/EBIT,EvOrdenado,ROICOrdenado,score,sector
0,BRBI11.SA,232.01,6.74,71,0,71,Financial Services
1,UNIP6.SA,57.45,4.18,29,1,30,Basic Materials
2,UNIP3.SA,57.45,3.88,25,2,27,Basic Materials
3,TASA3.SA,57.32,2.83,18,3,21,Industrials
4,TASA4.SA,57.32,2.82,17,4,21,Industrials
...,...,...,...,...,...,...,...
200,MGLU3.SA,0.95,122.27,200,200,400,Consumer Cyclical
201,LWSA3.SA,0.71,122.49,201,201,402,Technology
202,TPIS3.SA,0.43,154.51,202,202,404,Industrials
203,SEQL3.SA,0.34,309.53,204,203,407,Industrials


In [21]:
# Tipos de setores
dadosNovos['sector'].unique()

array(['Financial Services', 'Basic Materials', 'Industrials',
       'Consumer Defensive', 'Energy', 'Healthcare', 'Real Estate',
       'Consumer Cyclical', 'Communication Services', 'Technology',
       'Utilities', 'NA'], dtype=object)

In [22]:
filtroSetores = dadosNovos[dadosNovos['sector'] != 'Financial Services']
filtroSetores = filtroSetores[filtroSetores['sector'] != 'Energy']
filtroSetores = filtroSetores[filtroSetores['sector'] != 'Utilities']
filtroSetores = filtroSetores[filtroSetores['sector'] != 'NA']
filtroSetores

,Papel,ROIC,EV/EBIT,EvOrdenado,ROICOrdenado,score,sector
1,UNIP6.SA,57.45,4.18,29,1,30,Basic Materials
2,UNIP3.SA,57.45,3.88,25,2,27,Basic Materials
3,TASA3.SA,57.32,2.83,18,3,21,Industrials
4,TASA4.SA,57.32,2.82,17,4,21,Industrials
6,CMIN3.SA,50.79,2.77,16,6,22,Basic Materials
...,...,...,...,...,...,...,...
200,MGLU3.SA,0.95,122.27,200,200,400,Consumer Cyclical
201,LWSA3.SA,0.71,122.49,201,201,402,Technology
202,TPIS3.SA,0.43,154.51,202,202,404,Industrials
203,SEQL3.SA,0.34,309.53,204,203,407,Industrials


In [24]:
# Ranking da Magic Formula
filtroSetores.sort_values(by = 'score').head(30)

,Papel,ROIC,EV/EBIT,EvOrdenado,ROICOrdenado,score,sector
14,SYNE3.SA,37.58,1.05,2,14,16,Real Estate
11,BRKM5.SA,39.71,2.14,7,11,18,Basic Materials
9,MRFG3.SA,41.81,2.29,10,9,19,Consumer Defensive
18,GOAU4.SA,35.87,0.85,1,18,19,Basic Materials
3,TASA3.SA,57.32,2.83,18,3,21,Industrials
4,TASA4.SA,57.32,2.82,17,4,21,Industrials
6,CMIN3.SA,50.79,2.77,16,6,22,Basic Materials
8,VALE3.SA,42.49,2.65,14,8,22,Basic Materials
16,GGBR3.SA,35.92,2.15,9,16,25,Basic Materials
7,AURA33.SA,45.82,2.93,19,7,26,Basic Materials
